In [2]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
import os
os.getcwd()

'/Users/yezhengshao/School/ECE1786/final_project/GOTalk/train'

In [ ]:
## uncomment to run on colab GPU


# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
## uncomment to run on colab GPU

# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

# ! pip install datasets transformers
# !apt install git-lfs

In [2]:
# login to huggingface

from huggingface_hub import notebook_login

notebook_login()

In [6]:
file = open("../data/got1_fined.txt", "r")
full_texts =[]
texts = []
labels = []
for i in range(905):
    line = file.readline()
    words = word_tokenize(line)
    sentence = TreebankWordDetokenizer().detokenize(words)
    if len(sentence) == 0: 
        print(line, i)
    if sentence[-1] == ']':
        sentence = sentence + ' '
    sentence = sentence.replace(" ’ ", " ' ")
    full_texts.append(sentence)
    # texts.append(TreebankWordDetokenizer().detokenize(words[:-1]))
    # labels.append(TreebankWordDetokenizer().detokenize(words[1:]))
    
train, test = train_test_split(full_texts, test_size=0.2, shuffle=False)


file = open('../data/got_train.txt', 'w')
for i in train:
    file.write(i)
file.close()

file = open('../data/got_test.txt', 'w')
for i in test:
    file.write(i)
file.close()

In [3]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("HScomcom/gpt2-game-of-thrones", truncation=True)

SPECIAL_TOKENS = {
    "bos_token": "[BOS]",
    "eos_token": "[EOS]",
    "pad_token": "[PAD]"
    # "additional_special_tokens": 
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

# train_path = '../data/got_train.txt'
train_path = '../data/got3_fined.txt'
# test_path = '../data/got_test.txt'

#def load_dataset(train_path,test_path,tokenizer):
def load_dataset(train_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=100,
          )

    # test_dataset = TextDataset(
    #       tokenizer=tokenizer,
    #       file_path=test_path,
    #       block_size=100,
    #       )   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    # return train_dataset,test_dataset,data_collator
    return train_dataset,data_collator

# train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)
train_dataset,data_collator = load_dataset(train_path,tokenizer)



/Users/yezhengshao/opt/anaconda3/envs/ECE1786/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (80374 > 1024). Running this sequence through the model will result in indexing errors


In [16]:
train_dataset[100]

tensor([  780,   339,   564,   247,   264,  2330,   764,   383,  1854,   389,
          477,  3223,    11, 13791,   393,  2042,   764,   220, 50258,  1318])

In [6]:
tokenizer.decode(train_dataset[99])

' s too … it ’ s cold. [EOS] Yes. Cold and hard and mean, that ’ s the Wall, and the men who walk it. Not like the stories your wet nurse told you. Well, piss on the stories and piss on your wet nurse. This is the way it is, and you ’ re here for life, same as the rest of us.Jon repeated bitterly. The armorer could talk about life. He ’ d had one. He �'

In [7]:
tokenizer

PreTrainedTokenizerFast(name_or_path='HScomcom/gpt2-game-of-thrones', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '[PAD]'})

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("HScomcom/gpt2-game-of-thrones")

training_args = TrainingArguments(
    f"GPT2-GOT2-finetuned",
    # output_dir="/content", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    # per_device_eval_batch_size=4,  # batch size for evaluation
    # eval_steps = 1000, # Number of update steps between two evaluations.
    # evaluation_strategy="steps",
    save_steps=1000, # after # steps model is saved 
    warmup_steps=250,# number of warmup steps for learning rate scheduler
    push_to_hub=True # push to the huggingface
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,

)

trainer.train()
# trainer.save_model()

In [ ]:
# evaluate
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text = "Jon said [BOS]"
encoded_input = tokenizer(text, return_tensors='pt').input_ids
# input_ids = encoded_input.input_ids

outputs = model.generate(encoded_input.to(device), do_sample=True, max_length=200, temperature=1.1, top_p = 0.8, pad_token_id = 50256)
res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(res)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# text = "Jon grinned and reached under the table to ruffle the shaggy white fur. Inside, one of the guards looked at him and said"
text = 'Ygritte:"You know nothing, Jon Snow." Jon said'
text = "Ben clapped Jon on the shoulder. “Come, let’s take a walk in the garden. I’m sure you have much to tell me.” Jon said"
# text = 'Jon grinned and reached under the table to ruffle the shaggy white fur. Inside, one of the guards looked at him and said'
encoded_input = tokenizer(text, return_tensors='pt').input_ids

# beam search with force_words for generating dialogue options
force_words = ['[BOS]', '[EOS]']
force_words_ids = tokenizer(force_words, add_special_tokens=False).input_ids
outputs = model.generate(encoded_input.to(device), do_sample=False, max_length=100, temperature=1.6, top_p = 0.9, pad_token_id = 50256, num_beams=10,
                         repetition_penalty = 1, force_words_ids=force_words_ids,)


# for generating context 
# outputs = model.generate(encoded_input.to(device), do_sample=True, max_length=100, temperature=1, top_p = 1, pad_token_id = 50256,
#                          repetition_penalty = 1.1)


res = tokenizer.batch_decode(outputs, skip_speical_tokens=False)
print(res)

In [ ]:
# upload to huggingface
model.push_to_hub("GPT2-GOT4-finetuned")
tokenizer.push_to_hub("GPT2-GOT4-finetuned")
